In [1]:
import pandas as pd
import numpy as np

import math
import scipy.optimize as opt
import random
random.seed(108)


In [2]:
# Data: Close
db = pd.read_excel('Data.xlsx', index_col = "Date")

# MSD

In [3]:
db.index = pd.to_datetime(db.index)
MSD_df = db['2006':'2013']
MSD_df = (MSD_df-MSD_df.mean())/MSD_df.std() #change rolling
MSD_df = MSD_df.dropna(axis = 1) #drop nan

In [4]:
target = {}
for i in range(0,len(MSD_df.columns)-1):
    for j in range(i+1,len(MSD_df.columns)):
        tmp = ((MSD_df.iloc[:,i] - MSD_df.iloc[:,j])**2).sum()
        target[str(MSD_df.columns[i])+','+str(MSD_df.columns[j])] = tmp

In [5]:
target_df = pd.DataFrame.from_dict(target, orient='index')
target_df = target_df.sort_values(by = 0)[0:10]
target_df

,0
"DIS,TRV",106.019533
"IBM,MCD",174.553811
"AAPL,IBM",183.653680
"CVX,KO",205.031258
"IBM,KO",225.295065
"DIS,HD",264.348677
"HD,TRV",282.786437
"PEP,PG",298.596370
"DIS,MMM",300.548727
"GOOGL,TRV",304.718140


In [6]:
pair1_df_train = db.loc["2006":"2013",target_df.index[1].split(",")].pct_change(1).dropna()
pair1_df_test = db.loc["2014":"2014",target_df.index[1].split(",")].pct_change(1).dropna()
pair1_df_train

,IBM,MCD
Date,,
2006-01-04,-0.001340,0.008950
2006-01-05,0.006711,0.001183
2006-01-06,0.029697,0.005907
2006-01-09,-0.014361,0.019084
2006-01-10,0.004061,0.000864
...,...,...
2013-12-24,0.005433,0.002492
2013-12-26,0.011625,0.003107
2013-12-27,-0.001457,0.000723


In [7]:
# ytlist = pair1_df_train.iloc[:,0] - pair1_df_train.iloc[:,1]

In [8]:
from scipy.stats import t

# ytlist = [] #用來收集過去的yt們，算z_td、s_z時會用到

# #Table1 參數
# fi_1_0 = 0.1046
# fi_1_1 = 0.3886
# fi_2_0 = 0.0961
# fi_2_1 = -0.2403
# alpha_1_0 = 0.1639
# alpha_1_1 = 0.2132
# beta_1_1 = 0.6631
# alpha_2_0 = -0.1035
# alpha_2_1 = -0.1063
# beta_2_1 = -0.1751
# v = 7
# gamma = 5.5563
# c_1 = -0.3569
# c_2 = 0.3136
# d = 1
# z_td = yt[len(yt)-d:-1] #t-d
# s_z = std(z_td)

# Fdis = (1+np.exp(-gamma*(z_td - c_1)(z_td - c_2))/s_z)**(-1)
# h_1_t = alpha_1_0 + alpha_1_1*a_t1**2 + beta_1_1*h_t1
# h_2_t = alpha_2_0 + alpha_2_1*a_t1**2 + beta_2_1*h_t1
# miu_1 = fi_1_0 + fi_1_1*y_t1
# miu_2 = fi_2_0 + fi_2_1*y_t1
# h_t = h_1_t + Fdis*h_2_t
# epsilon = t.rvs(v, size=1) #t-distribution
# a_t = h_t**0.5*epsilon #error term
# y_t = miu_1 + Fdis*miu_2 + a_t #ST-GARCH


In [9]:
z_td = pair1_df_train.iloc[0:30,0]-pair1_df_train.iloc[0:30,1]
z_td

Date
2006-01-04   -0.010290
2006-01-05    0.005529
2006-01-06    0.023790
2006-01-09   -0.033445
2006-01-10    0.003196
2006-01-11   -0.016082
2006-01-12    0.008435
2006-01-13    0.004412
2006-01-17   -0.005525
2006-01-18   -0.001926
2006-01-19   -0.014474
2006-01-20   -0.039571
2006-01-23    0.004797
2006-01-24   -0.010799
2006-01-25    0.019989
2006-01-26    0.003909
2006-01-27    0.000568
2006-01-30    0.008385
2006-01-31   -0.003757
2006-02-01   -0.010980
2006-02-02   -0.002497
2006-02-03   -0.030180
2006-02-06   -0.010200
2006-02-07    0.002616
2006-02-08    0.007200
2006-02-09   -0.003300
2006-02-10    0.010741
2006-02-13   -0.011769
2006-02-14    0.007806
2006-02-15    0.006939
dtype: float64

In [10]:
# fi_1_0, fi_1_1, fi_2_0, fi_2_1, alpha_1_0, alpha_1_1, beta_1_1, alpha_2_0, alpha_2_1, beta_2_1, v, gamma, c_1, c_2 = np.array([0.1046, 0.3886, 0.0961, -0.2403, 0.1639, 0.2132, 0.6631, -0.1035, -0.1063, -0.1751, 7.1122, 5.5563, -0.3569, 0.3136])
# tmp = 30
# z_td = pair1_df_train.iloc[0:tmp,0] - pair1_df_train.iloc[0:tmp,1]
# s_z = z_td.std()   
# Fdis = (1+np.exp(-gamma*(z_td - c_1)*(z_td - c_2)/s_z))**(-1)
# Fdis

In [11]:
def fun_MLE(x):
    fi_1_0, fi_1_1, fi_2_0, fi_2_1, alpha_1_0, alpha_1_1, beta_1_1, alpha_2_0, alpha_2_1, beta_2_1, gamma, c_1, c_2= x
    # Define h_t1
    tmp = 30
    target = 0
    z_td = pair1_df_train.iloc[0:tmp,0]-pair1_df_train.iloc[0:tmp,1]
    z_td = z_td.to_numpy()
    s_z = z_td.std()
    Fdis = (1+np.exp(-gamma*(z_td[-1] - c_1)*(z_td[-1] - c_2)/s_z))**(-1)
    h_1_t = pair1_df_train.iloc[0:tmp,0].std()
    h_2_t = pair1_df_train.iloc[0:tmp,1].std()
    h_t1 = h_1_t + Fdis*h_2_t
    epsilon = random.gauss(mu = 0, sigma = 1)
    a_t1 = h_t1**0.5*epsilon #error term


    y_t1 = pair1_df_train.iloc[tmp-1,0] - pair1_df_train.iloc[tmp-1,1]
    # Define sigma, real_y_t, y_t
    for i in range(tmp,len(pair1_df_train)):
        z_td = pair1_df_train.iloc[0:i,0]-pair1_df_train.iloc[0:i,1]
        z_td = z_td.to_numpy()
        s_z = z_td.std()
        real_y_t = pair1_df_train.iloc[i,0] - pair1_df_train.iloc[i,1]
        # real_y_t = real_y_t.to_numpy()
        h_1_t = alpha_1_0 + alpha_1_1*a_t1**2 + beta_1_1*h_t1
        h_2_t = alpha_2_0 + alpha_2_1*a_t1**2 + beta_2_1*h_t1
        miu_1 = fi_1_0 + fi_1_1*y_t1
        miu_2 = fi_2_0 + fi_2_1*y_t1
        h_t = h_1_t + Fdis*h_2_t
        epsilon = random.gauss(mu = 0, sigma = 1)
        a_t = h_t**0.5*epsilon #error term
        y_t = miu_1 + Fdis*miu_2 + a_t #ST-GARCH

        h_t1 = h_t
        a_t1 = a_t
        y_t1 = real_y_t

        sigma = h_t
        target += np.log((2*np.pi*sigma**2)**(-0.5)*np.exp(-(real_y_t-y_t)**2/(2*sigma**2))) #MLE
        # print(target)
    target = -target
    
    return target

In [12]:
#MLE
tmp = 1e-9*4
initial = np.array([0.1068, 0.3838, 0.0940, -0.2366, 0.1649, 0.2062, 0.6622, -0.1045, -0.0981, -0.1827, 4.9262, -0.3516, 0.3068]) 
b1 = ((pair1_df_train.iloc[0:30,0] - pair1_df_train.iloc[0:30,1]).std())**2*10000
b2 = 1
b3 = 1.1
constraints = ( {'type':'ineq','fun':lambda x: x[4]+tmp},
                {'type':'ineq','fun':lambda x: x[5]+tmp},
                {'type':'ineq','fun':lambda x: x[6]+tmp},
                {'type':'ineq','fun':lambda x: x[5]+x[8]+tmp},
                {'type':'ineq','fun':lambda x: x[6]+x[9]+tmp},
                {'type':'ineq','fun':lambda x:-x[5]-0.5*x[8]-x[6]-0.5*x[9]+1+tmp},
                {'type':'ineq','fun':lambda x:-x[4]+b1+tmp},     #b1
                {'type':'ineq','fun':lambda x:-x[6]+b2+tmp},     #b2 = 1
                {'type':'ineq','fun':lambda x:-x[5]-x[6]+b3+tmp},#b3 = 1.1
                {'type':'ineq','fun':lambda x: x[12]-x[11]+tmp} 
                  )
bound = (-0.9,0.9)
# bounds = tuple(bound for asset in range(len(initial)))
bounds = ((-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-5, 5),
 (-0.9, 0.9),
 (-0.9, 0.9))
minimum = opt.minimize(fun_MLE, initial, constraints = constraints, method = 'trust-constr', bounds = bounds) #bounds: 0.9
minimum

/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:186: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 1985
      cg_stop_cond: 2
            constr: [array([0.35547014]), array([0.06975327]), array([0.36181353]), array([0.40395467]), array([0.24222671]), array([0.46112592]), array([1.69767655]), array([0.63818647]), array([0.66843321]), array([0.77812378]), array([-0.18365375,  0.29991262,  0.08903568, -0.38796593,  0.35547014,
        0.06975327,  0.36181353,  0.25890811,  0.3342014 , -0.11958682,
        4.46605713, -0.33643076,  0.44169301])]
       constr_nfev: [6552, 6552, 6552, 6552, 6552, 6552, 6552, 6552, 6552, 6552, 0]
       constr_nhev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 2865.2162334918976
               fun: 2473.701933459208
              grad: array([-1.83855603e+09,  5.25685949e+09,  2.05015779e+08, -4.45901069e+09,
        1.00164169e+10,  4.7803090

In [13]:
minimum.x

array([-0.18365375,  0.29991262,  0.08903568, -0.38796593,  0.35547014,
        0.06975327,  0.36181353,  0.25890811,  0.3342014 , -0.11958682,
        4.46605713, -0.33643076,  0.44169301])

# DIS, TRV

In [14]:
minimum.x

array([-0.18365375,  0.29991262,  0.08903568, -0.38796593,  0.35547014,
        0.06975327,  0.36181353,  0.25890811,  0.3342014 , -0.11958682,
        4.46605713, -0.33643076,  0.44169301])

# Trade

In [15]:
df_return = db.pct_change()

In [16]:
global results

In [17]:
results = {'signals':[],
           'returns':[],
           'A_SoldPrices':[],
           'A_BoughtPrices':[],
           'B_SoldPrices':[],
           'B_BoughtPrices':[]
           }

In [18]:
# Input1 : Real return spread
# Input2 : Estimated return spread
# Output : Gain, Trading Signal

y1 = df_return.iloc[-1,0] 
y2 = df_return.iloc[-1,1]
p1 = db.iloc[-1:,0] #A Price
p2 = db.iloc[-1,1] #B Price

#First trade
i = 0

#signal
tmp0 = 0
tmp1 = 1
tmp2 = -1

#First trade
if i == 0:
    if (y1 > y2):
        signal = tmp1
        results['A_SoldPrices'].append(p1)
        results['B_BoughtPrices'].append(p2)
        i = 1
        tmp = 0
    
    elif (y1 < y2):  
        signal = tmp2
        results['A_BoughtPrices'].append(p1)
        results['B_SoldPrices'].append(p2) 
        i = 1
        tmp = 0
    else:
        signal = tmp0
        i = 0
        tmp = 1

elif i == 1:
    # After first trade
    if (y1 > y2) & (results['signals'][-1] == tmp1) :
        signal = tmp1
        results['A_SoldPrices'].append(y1)
        results['B_BoughtPrices'].append(y2)
        tmp = -np.log(results['A_SoldPrices'][-1]/results['A_BoughtPrices'][-1])+np.log(results['B_SoldPrices'][-1]/results['B_BoughtPrices'][-1])
  
    elif (y1 <y2) & (results['signals'][-1] == tmp2):  
        signal = tmp2
        results['A_BoughtPrices'].append(y1)
        results['B_SoldPrices'].append(y2)
        tmp = np.log(results['A_SoldPrices'][-1]/results['A_BoughtPrices'][-1])-np.log(results['B_SoldPrices'][-1]/results['B_BoughtPrices'][-1]) 

    else:
        signal = tmp0
        tmp = 0


results['signals'].append(signal)
results['returns'].append(tmp)